# Explore files

In [1]:
import os
import pandas as pd

# Specify the directory containing the CSV files
directory = 'data'

# Loop through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        # Print the DataFrame
        print(f'Contents of {filename}:')
        print(df)
        print('\n')


Contents of checkin_checkout_history_updated.csv:
          user_id  gym_id         checkin_time        checkout_time  \
0       user_3291   gym_6  2023-09-10 15:55:00  2023-09-10 16:34:00   
1       user_1944   gym_2  2023-04-13 20:07:00  2023-04-13 22:43:00   
2        user_958   gym_7  2023-06-10 12:24:00  2023-06-10 13:49:00   
3        user_811   gym_2  2023-05-23 17:11:00  2023-05-23 20:01:00   
4       user_4923  gym_10  2023-02-21 06:20:00  2023-02-21 08:02:00   
...           ...     ...                  ...                  ...   
299995  user_3995   gym_3  2023-08-06 17:25:00  2023-08-06 18:09:00   
299996   user_206   gym_9  2023-06-27 13:14:00  2023-06-27 16:04:00   
299997  user_4983   gym_4  2023-04-08 14:41:00  2023-04-08 15:54:00   
299998  user_1028  gym_10  2023-03-05 06:07:00  2023-03-05 07:04:00   
299999  user_3314   gym_4  2023-01-05 08:58:00  2023-01-05 09:48:00   

         workout_type  calories_burned  
0       Weightlifting              462  
1              

# Data Modeling

### Normalized (OLTP) schema
Designed to reduce redundancy and ensure data integrity by organizing the data into multiple related tables.

1. **Users**
   - **user_id_pk** (PK)
   - **fk_subscription_plan_id** (FK)
   - **first_name**
   - **last_name**
   - **age**
   - **gender**
   - **birthdate**
   - **sign_up_date**
   - **user_location**

2. **Gyms**
   - **gym_id_pk** (PK)
   - **location**
   - **gym_type**
   - **facilities**

3. **Subscription Plans**
   - **subscription_plan_id_pk** (PK)
   - **plan_name**
   - **price_per_month**
   - **features**

4. **Check-ins**
   - **checkin_id_pk** (PK)
   - **fk_user_id** (FK)
   - **fk_gym_id** (FK)
   - **checkin_time**
   - **checkout_time**
   - **workout_type**
   - **calories_burned**

#### Relationships
- **Users** to **Subscription Plans**:  One-to-Many
- **Check-ins** to **Users**:  Many-to-One
- **Check-ins** to **Gyms**:  Many-to-One

#### Schema Definition

```sql
CREATE TABLE users (
    user_id_pk INT PRIMARY KEY,
    fk_subscription_plan_id INT,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    age INT,
    gender VARCHAR(10),
    birthdate DATE,
    sign_up_date DATE,
    user_location VARCHAR(100),
    FOREIGN KEY (fk_subscription_plan_id) REFERENCES subscription_plans(subscription_plan_id_pk)
);

CREATE TABLE gyms (
    gym_id_pk INT PRIMARY KEY,
    location VARCHAR(100),
    gym_type VARCHAR(50),
    facilities TEXT
);

CREATE TABLE subscription_plans (
    subscription_plan_id_pk INT PRIMARY KEY,
    plan_name VARCHAR(50),
    price_per_month DECIMAL(10, 2),
    features TEXT
);

CREATE TABLE checkins (
    checkin_id_pk SERIAL PRIMARY KEY,
    fk_user_id INT,
    fk_gym_id INT,
    checkin_time TIMESTAMP,
    checkout_time TIMESTAMP,
    workout_type VARCHAR(50),
    calories_burned INT,
    FOREIGN KEY (fk_user_id) REFERENCES users(user_id_pk),
    FOREIGN KEY (fk_gym_id) REFERENCES gyms(gym_id_pk)
);
```


### Star schema
Simplifies complex queries with a central fact table connected to dimension tables.

**Fact Table: Check-in/Checkout History**
- **fk_user_id** (FK)
- **fk_gym_id** (FK)
- **checkin_time**
- **checkout_time**
- **workout_type**
- **calories_burned**

**Dimension Tables:**

1. **Users**
   - **user_id_pk** (PK)
   - **first_name**
   - **last_name**
   - **age**
   - **gender**
   - **birthdate**
   - **sign_up_date**
   - **user_location**
   - **subscription_plan_name**

2. **Gyms**
   - **gym_id_pk** (PK)
   - **location**
   - **gym_type**
   - **facilities**

3. **Subscription Plans**
   - **subscription_plan_pk** (PK)
   - **plan_name**
   - **price_per_month**
   - **features**

#### Schema definition

```sql
CREATE TABLE users (
    user_id_pk INT PRIMARY KEY,
    subscription_plan_id INT,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    age INT,
    gender VARCHAR(10),
    birthdate DATE,
    sign_up_date DATE,
    user_location VARCHAR(100)
);

CREATE TABLE gyms (
    gym_id_pk INT PRIMARY KEY,
    location VARCHAR(100),
    gym_type VARCHAR(50),
    facilities TEXT
);

CREATE TABLE subscription_plans (
    subscription_plan_id_pk INT PRIMARY KEY,
    plan_name VARCHAR(50),
    price_per_month DECIMAL(10, 2),
    features TEXT
);

CREATE TABLE checkins (
    fk_user_id INT,
    fk_gym_id INT,
    checkin_time TIMESTAMP,
    checkout_time TIMESTAMP,
    workout_type VARCHAR(50),
    calories_burned INT
);

```


### Snowflake schema
Normalized form of the star schema, reducing redundancy.

1. **Fact Table**: Central table containing quantitative data.
2. **Dimension Tables**: Surrounding tables containing descriptive attributes, further normalized into sub-dimension tables.

**Fact Table: Check-ins**
- **checkin_id_pk** (PK)
- **fk_user_id** (FK)
- **fk_gym_id** (FK)
- **checkin_time**
- **checkout_time**
- **workout_type**
- **calories_burned**

**Dimension Tables**

1. **Users**
   - **user_id_pk** (PK)
   - **fk_subscription_plan_id** (FK)
   - **first_name**
   - **last_name**
   - **age**
   - **gender**
   - **birthdate**
   - **sign_up_date**
   - **user_location**

2. **Gyms**
   - **gym_id_pk** (PK)
   - **fk_location_id** (FK)
   - **gym_type**
   - **facilities**

3. **Subscription Plans**
   - **subscription_plan_id** (PK)
   - **plan_name**
   - **price_per_month**
   - **features**

**Sub-Dimension Tables**

1. **Locations**
   - **location_id_pk** (PK)
   - **location_name**
   - **address**
   - **city**
   - **state**
   - **zip_code**

### Relationships

- **Users** to **Subscription Plans**: One-to-Many
- **Gyms** to **Locations**: Many-to-One
- **Check-ins** to **Users**: Many-to-One
- **Check-ins** to **Gyms**: Many-to-One

#### Schema definition

```sql
CREATE TABLE users (
    user_id_pk INT PRIMARY KEY,
    fk_subscription_plan_id INT,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    age INT,
    gender VARCHAR(10),
    birthdate DATE,
    sign_up_date DATE,
    user_location VARCHAR(100),
    FOREIGN KEY (fk_subscription_plan_id) REFERENCES subscription_plans(subscription_plan_id_pk)
);

CREATE TABLE gyms (
    gym_id_pk INT PRIMARY KEY,
    location_id VARCHAR(100),
    gym_type VARCHAR(50),
    facilities TEXT
);

CREATE TABLE subscription_plans (
    subscription_plan_id_pk INT PRIMARY KEY,
    plan_name VARCHAR(50),
    price_per_month DECIMAL(10, 2),
    features TEXT
);

CREATE TABLE checkins (
    checkin_id_pk SERIAL PRIMARY KEY,
    fk_user_id INT,
    fk_gym_id INT,
    checkin_time TIMESTAMP,
    checkout_time TIMESTAMP,
    workout_type VARCHAR(50),
    calories_burned INT,
    FOREIGN KEY (fk_user_id) REFERENCES users(user_id_pk),
    FOREIGN KEY (fk_gym_id) REFERENCES gyms(gym_id_pk)
);

```



## Postgres database build script.
From **psql**, execute the created *db_build.sql* file

In [2]:
## Postgres database build script.

db_build_script = r"""
SELECT pg_terminate_backend(pid)
FROM pg_stat_activity
WHERE datname = 'csv_oltp';

DROP DATABASE IF EXISTS csv_oltp;
CREATE DATABASE csv_oltp;
\connect csv_oltp;
CREATE SCHEMA schema_oltp;

CREATE TABLE schema_oltp.gyms (
    gym_id_pk INT PRIMARY KEY,
    location VARCHAR(100),
    gym_type VARCHAR(50),
    facilities TEXT
);

CREATE TABLE schema_oltp.subscription_plans (
    subscription_plan_id_pk INT PRIMARY KEY,
    plan_name VARCHAR(50),
    price_per_month DECIMAL(10, 2),
    features TEXT
);

CREATE TABLE schema_oltp.users (
    user_id_pk INT PRIMARY KEY,
    fk_subscription_plan_id INT,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    age INT,
    gender VARCHAR(10),
    birthdate DATE,
    sign_up_date DATE,
    user_location VARCHAR(100),
    FOREIGN KEY (fk_subscription_plan_id) REFERENCES schema_oltp.subscription_plans(subscription_plan_id_pk)
);

CREATE TABLE schema_oltp.checkins (
    checkin_id_pk SERIAL PRIMARY KEY,
    fk_user_id INT,
    fk_gym_id INT,
    checkin_time TIMESTAMP,
    checkout_time TIMESTAMP,
    workout_type VARCHAR(50),
    calories_burned INT,
    FOREIGN KEY (fk_user_id) REFERENCES schema_oltp.users(user_id_pk),
    FOREIGN KEY (fk_gym_id) REFERENCES schema_oltp.gyms(gym_id_pk)
);


SELECT pg_terminate_backend(pid)
FROM pg_stat_activity
WHERE datname = 'csv_star';

DROP DATABASE IF EXISTS csv_star;
CREATE DATABASE csv_star;
\connect csv_star;
CREATE SCHEMA schema_star;

CREATE TABLE schema_star.users (
    user_id_pk INT PRIMARY KEY,
    subscription_plan_name VARCHAR(50),
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    age INT,
    gender VARCHAR(10),
    birthdate DATE,
    sign_up_date DATE,
    user_location VARCHAR(100)
);

CREATE TABLE schema_star.gyms (
    gym_id_pk INT PRIMARY KEY,
    location VARCHAR(100),
    gym_type VARCHAR(50),
    facilities TEXT
);

CREATE TABLE schema_star.subscription_plans (
    subscription_plan_id_pk INT PRIMARY KEY,
    plan_name VARCHAR(50),
    price_per_month DECIMAL(10, 2),
    features TEXT
);

CREATE TABLE schema_star.checkins (
    fk_user_id INT,
    fk_gym_id INT,
    checkin_time TIMESTAMP,
    checkout_time TIMESTAMP,
    workout_type VARCHAR(50),
    calories_burned INT
);


SELECT pg_terminate_backend(pid)
FROM pg_stat_activity
WHERE datname = 'csv_snow';

DROP DATABASE IF EXISTS csv_snow;
CREATE DATABASE csv_snow;
\connect csv_snow;
CREATE SCHEMA schema_snow;

CREATE TABLE schema_snow.subscription_plans (
    subscription_plan_id_pk INT PRIMARY KEY,
    plan_name VARCHAR(50),
    price_per_month DECIMAL(10, 2),
    features TEXT
);

CREATE TABLE schema_snow.users (
    user_id_pk INT PRIMARY KEY,
    fk_subscription_plan_id INT,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    age INT,
    gender VARCHAR(10),
    birthdate DATE,
    sign_up_date DATE,
    user_location VARCHAR(100),
    FOREIGN KEY (fk_subscription_plan_id) REFERENCES schema_snow.subscription_plans(subscription_plan_id_pk)
);

CREATE TABLE schema_snow.gyms (
    gym_id_pk INT PRIMARY KEY,
    location VARCHAR(100),
    gym_type VARCHAR(50),
    facilities TEXT
);

CREATE TABLE schema_snow.checkins (
    checkin_id_pk SERIAL PRIMARY KEY,
    fk_user_id INT,
    fk_gym_id INT,
    checkin_time TIMESTAMP,
    checkout_time TIMESTAMP,
    workout_type VARCHAR(50),
    calories_burned INT,
    FOREIGN KEY (fk_user_id) REFERENCES schema_snow.users(user_id_pk),
    FOREIGN KEY (fk_gym_id) REFERENCES schema_snow.gyms(gym_id_pk)
);

"""

with open("db_build.sql", "w") as file:
    file.write(db_build_script)


A database build file, **db_build.sql**, was created.  Enter **psql**, then execute the file by entering:

```sql
\i /path/to/db_build.sql
```


In [3]:
# Wait for a key press
input("Build DB then press Enter to continue...")

''

In [4]:
# define database connection parameters
DB_USER = "postgres"
DB_PASSWORD = "abcd1234"
DB_HOST = "192.168.50.75"
DB_PORT = "5432"


In [5]:
import psycopg2

def get_connection(db):
    # Database connection parameters
    conn = psycopg2.connect(       
        dbname=db,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )

    return conn, conn.cursor()


In [6]:
def display_count(db, table_name):
    conn, cur = get_connection(db)

    cur.execute(f"SELECT COUNT(*) FROM {table_name};")
    count = cur.fetchone()[0]

    print(f"{table_name} row count: {count}")

    # Close connection
    cur.close()
    conn.close()



In [7]:
def get_random_rows_sql(table_name, n=3):
    sql = f"SELECT * FROM {table_name} ORDER BY RANDOM() LIMIT {n};"
    return sql

In [8]:
from sqlalchemy import text


def display_results(engine, sql):
    with engine.connect() as conn:
        print(conn.execute(text(sql)).fetchall())


In [9]:
from sqlalchemy import create_engine

def get_engine(db):
    # Database connection details
    username = DB_USER
    password = DB_PASSWORD
    host = DB_HOST
    port = DB_PORT
    database = db

    # Create a SQLAlchemy engine
    return create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')


## Cleanup OLTP Schema
Since this is the first data load for the tables, confirm that they are currently unpopulated.

In [10]:
conn, cur = get_connection('csv_oltp')

# List of DELETE statements
delete_statements = [
    "DELETE FROM schema_oltp.checkins CASCADE;",
    "DELETE FROM schema_oltp.users CASCADE;",
    "DELETE FROM schema_oltp.gyms CASCADE;",
    "DELETE FROM schema_oltp.subscription_plans CASCADE;",
]

# Execute each DELETE statement
for statement in delete_statements:
    cur.execute(statement)

conn.commit()
cur.close()
conn.close()


## Load OTLP schema
- gyms
    - The index value is prefixed with "gym_", remove it.
- subscription_plans
- users
    - The index value is prefixed with "user_", remove it.
- checkins

#### Load gym table

In [11]:
import pandas as pd
from sqlalchemy import text

# read file
file_path = 'data/gym_locations_data.csv'

df = pd.read_csv(file_path)
# extract gym_id_pk, the index value
df['gym_id_pk'] = df['gym_id'].str.extract(r'(\d+)').astype(int)
# reorder columns to match table
df = df[['gym_id_pk', 'location',  'gym_type', 'facilities']]

# Create a SQLAlchemy engine
engine = get_engine('csv_oltp')

# Insert DataFrame into PostgreSQL table
df.to_sql('gyms', engine, schema='schema_oltp', if_exists='append', index=False)
display_results(engine, get_random_rows_sql("schema_oltp.gyms"))
display_count("csv_oltp", "schema_oltp.gyms")



[(3, 'Chicago', 'Budget', 'Sauna, Climbing Wall, Swimming Pool'), (7, 'San Antonio', 'Premium', 'Sauna, Basketball Court, Swimming Pool'), (1, 'New York', 'Premium', 'Climbing Wall, Swimming Pool, Basketball Court')]
schema_oltp.gyms row count: 10


#### Load subscription_plans table

In [12]:
import pandas as pd
from sqlalchemy import text

# read file
file_path = 'data/subscription_plans.csv'

df = pd.read_csv(file_path)
df.reset_index(inplace=True)
df.rename(columns={'index': 'subscription_plan_id_pk'}, inplace=True)
df['subscription_plan_id_pk'] = df.index + 1
df.rename(columns={'subscription_plan' : 'plan_name'}, inplace=True)

# Create a SQLAlchemy engine
engine = get_engine('csv_oltp')

# Insert DataFrame into PostgreSQL table
df.to_sql('subscription_plans', engine, schema='schema_oltp', if_exists='append', index=False)
display_results(engine, get_random_rows_sql("schema_oltp.subscription_plans"))
display_count("csv_oltp", "schema_oltp.subscription_plans")



[(2, 'Pro', Decimal('49.99'), 'Access to all facilities, Unlimited class access, 5 guest passes per month, Free personal trainer session'), (1, 'Basic', Decimal('19.99'), 'Access to basic gym facilities, Limited class access, 1 guest pass per month'), (3, 'Student', Decimal('9.99'), 'Access to basic facilities, Limited class access, 1 guest pass per month, Discounted for students')]
schema_oltp.subscription_plans row count: 3


#### Load users table

In [13]:
import pandas as pd
from sqlalchemy import text

# read file
file_path = 'data/users_data.csv'

df = pd.read_csv(file_path)
# extract user_id_pk, the index value
df['user_id_pk'] = df['user_id'].str.extract(r'(\d+)').astype(int)
plan_mapping = {'Basic': 1, 'Pro': 2, 'Student': 3}
df['fk_subscription_plan_id'] = df['subscription_plan'].map(plan_mapping)

# reorder columns to match table
df = df[['user_id_pk', 'fk_subscription_plan_id',  'first_name', 'last_name', 'age', 'gender', 'birthdate', 'sign_up_date', 'user_location']]

# Create a SQLAlchemy engine
engine = get_engine('csv_oltp')

# Insert DataFrame into PostgreSQL table
df.to_sql('users', engine, schema='schema_oltp', if_exists='append', index=False)
display_results(engine, get_random_rows_sql("schema_oltp.users"))
display_count("csv_oltp", "schema_oltp.users")


[(4557, 1, 'Laura', 'Jones', 60, 'Female', datetime.date(1979, 12, 27), datetime.date(2023, 3, 4), 'Orlando'), (4455, 2, 'Jane', 'Brown', 35, 'Male', datetime.date(1972, 10, 19), datetime.date(2022, 8, 6), 'Denver'), (4964, 3, 'Michael', 'Davis', 29, 'Female', datetime.date(1992, 4, 13), datetime.date(2022, 11, 1), 'San Francisco')]
schema_oltp.users row count: 5000


#### Load checkins table

In [14]:
import pandas as pd
from sqlalchemy import text

# read file
file_path = 'data/checkin_checkout_history_updated.csv'

df = pd.read_csv(file_path)
# extract user_id_pk, the index value
df['fk_user_id'] = df['user_id'].str.extract(r'(\d+)').astype(int)
df['fk_gym_id'] = df['gym_id'].str.extract(r'(\d+)').astype(int)

# reorder columns to match table
df = df[['fk_user_id', 'fk_gym_id',  'checkin_time', 'checkout_time', 'workout_type', 'calories_burned']]

# Create a SQLAlchemy engine
engine = get_engine('csv_oltp')

# Insert DataFrame into PostgreSQL table
df.to_sql('checkins', engine, schema='schema_oltp', if_exists='append', index=False)
display_results(engine, get_random_rows_sql("schema_oltp.checkins"))
display_count("csv_oltp", "schema_oltp.checkins")



[(357770, 3392, 6, datetime.datetime(2023, 5, 8, 6, 46), datetime.datetime(2023, 5, 8, 7, 21), 'Yoga', 1429), (352724, 2921, 5, datetime.datetime(2023, 6, 22, 9, 46), datetime.datetime(2023, 6, 22, 12, 25), 'Pilates', 1624), (359617, 3917, 3, datetime.datetime(2023, 8, 30, 12, 0), datetime.datetime(2023, 8, 30, 12, 36), 'Pilates', 1926)]
schema_oltp.checkins row count: 300000


We have the option to replicate the method used for OLTP schemas by employing Python scripts for each OLAP table. However, considering the OLTP tables are already in place, a direct database-to-database transfer would be more efficient and faster. Therefore, crafting a set of SQL scripts to populate the OLAP tables would be the preferable approach.

## Cleanup OLAP Star Schema

In [15]:
conn, cur = get_connection('csv_star')

# List of DELETE statements
delete_statements = [
    "DELETE FROM schema_star.checkins;",
    "DELETE FROM schema_star.users;",
    "DELETE FROM schema_star.gyms;",
    "DELETE FROM schema_star.subscription_plans;",
]

# Execute each DELETE statement
for statement in delete_statements:
    cur.execute(statement)

conn.commit()
cur.close()
conn.close()

## Load OLAP Star schema
- users
- gyms
- subscription_plans
- checkins

In the OLAP Star schema, the absence of foreign key relationships means that the sequence of tables is inconsequential. This design allows for a more flexible approach to data analysis, as the structure does not impose a hierarchy on the tables, enabling various ways to query and manage the data without concern for table arrangement.

In [16]:
# OLAP Star SQL statements 
# These commands are designed to retrieve data from OLTP databases as 
# a preliminary step before populating the corresponding OLAP tables.

users_load_star_sql = """
SELECT user_id_pk
       , sp.plan_name AS subscription_plan_name
       , first_name
       , last_name
       , age
       , gender
       , birthdate
       , sign_up_date
       , user_location
FROM schema_oltp.users u
     , schema_oltp.subscription_plans sp
WHERE u.fk_subscription_plan_id = sp.subscription_plan_id_pk;
"""

gyms_load_star_sql = """
SELECT gym_id_pk
	   , location
	   , gym_type 
	   , facilities
FROM schema_oltp.gyms;
"""

subscription_plans_load_star_sql = """
SELECT subscription_plan_id_pk 
	   , plan_name 
	   , price_per_month 
	   , features
FROM schema_oltp.subscription_plans;
"""

checkins_star_load_sql = """
SELECT fk_user_id 
       , fk_gym_id 
       , checkin_time 
       , checkout_time 
       , workout_type 
       , calories_burned 
FROM schema_oltp.checkins;
"""


In [17]:
engine_src = get_engine('csv_oltp')
engine = get_engine('csv_star')

# load users table
df = pd.read_sql_query(users_load_star_sql, con=engine_src)
df.to_sql('users', engine, schema='schema_star', if_exists='append', index=False)
display_results(engine, get_random_rows_sql("schema_star.users"))
display_count("csv_star", "schema_star.users")

# load gyms table
df = pd.read_sql_query(gyms_load_star_sql, con=engine_src)
df.to_sql('gyms', engine, schema='schema_star', if_exists='append', index=False)
display_results(engine, get_random_rows_sql("schema_star.gyms"))
display_count("csv_star", "schema_star.gyms")

# load subscription_plans table
df = pd.read_sql_query(subscription_plans_load_star_sql, con=engine_src)
df.to_sql('subscription_plans', engine, schema='schema_star', if_exists='append', index=False)
display_results(engine, get_random_rows_sql("schema_star.subscription_plans"))
display_count("csv_star", "schema_star.subscription_plans")

# load checkins table
df = pd.read_sql_query(checkins_star_load_sql, con=engine_src)
df.to_sql('checkins', engine, schema='schema_star', if_exists='append', index=False)
display_results(engine, get_random_rows_sql("schema_star.checkins"))
display_count("csv_star", "schema_star.checkins")


[(4175, 'Student', 'Daniel', 'Garcia', 52, 'Male', datetime.date(1959, 12, 19), datetime.date(2022, 4, 11), 'Boston'), (1686, 'Basic', 'Jessica', 'Moore', 49, 'Female', datetime.date(1978, 4, 13), datetime.date(2022, 7, 22), 'Seattle'), (546, 'Basic', 'Chris', 'Smith', 56, 'Non-binary', datetime.date(1966, 6, 22), datetime.date(2021, 8, 2), 'Austin')]
schema_star.users row count: 5000
[(7, 'San Antonio', 'Premium', 'Sauna, Basketball Court, Swimming Pool'), (2, 'Los Angeles', 'Budget', 'Climbing Wall, Yoga Classes, Sauna'), (9, 'Dallas', 'Premium', 'Sauna, CrossFit, Yoga Classes')]
schema_star.gyms row count: 10
[(1, 'Basic', Decimal('19.99'), 'Access to basic gym facilities, Limited class access, 1 guest pass per month'), (3, 'Student', Decimal('9.99'), 'Access to basic facilities, Limited class access, 1 guest pass per month, Discounted for students'), (2, 'Pro', Decimal('49.99'), 'Access to all facilities, Unlimited class access, 5 guest passes per month, Free personal trainer sessi

## Cleanup OLAP Snow Schema

In [18]:
conn, cur = get_connection('csv_snow')

# List of DELETE statements
delete_statements = [
    "DELETE FROM schema_snow.checkins;",
    "DELETE FROM schema_snow.gyms;",
    "DELETE FROM schema_snow.users;",
    "DELETE FROM schema_snow.subscription_plans;",
]

# Execute each DELETE statement
for statement in delete_statements:
    cur.execute(statement)

conn.commit()
cur.close()
conn.close()

## Load OLAP Snow schema
- subscription_plans
- users
- gyms
- checkins

In [19]:
# OLAP Snow SQL Statements
subscription_plans_load_snow_sql = """ 
SELECT subscription_plan_id_pk 
	   , plan_name 
	   , price_per_month 
	   , features
FROM schema_oltp.subscription_plans;
"""

users_load_snow_sql = """ 
SELECT user_id_pk 
       , fk_subscription_plan_id 
       , first_name 
       , last_name 
       , age 
       , gender 
       , birthdate 
       , sign_up_date 
       , user_location 
FROM schema_oltp.users;
"""

gyms_load_snow_sql = """ 
SELECT gym_id_pk
	   , location
	   , gym_type 
	   , facilities
FROM schema_oltp.gyms;
"""

checkins_load_snow_sql = """ 
SELECT fk_user_id 
       , fk_gym_id 
       , checkin_time 
       , checkout_time 
       , workout_type 
       , calories_burned 
FROM schema_oltp.checkins;
"""


In [20]:
engine_src = get_engine('csv_oltp')
engine = get_engine('csv_snow')

# load subscription_plans table
df = pd.read_sql_query(subscription_plans_load_snow_sql, con=engine_src)
df.to_sql('subscription_plans', engine, schema='schema_snow', if_exists='append', index=False)
display_results(engine, get_random_rows_sql("schema_snow.subscription_plans"))
display_count("csv_snow", "schema_snow.subscription_plans")

# load users table
df = pd.read_sql_query(users_load_snow_sql, con=engine_src)
df.to_sql('users', engine, schema='schema_snow', if_exists='append', index=False)
display_results(engine, get_random_rows_sql("schema_snow.users"))
display_count("csv_snow", "schema_snow.users")

# load gyms table
df = pd.read_sql_query(gyms_load_snow_sql, con=engine_src)
df.to_sql('gyms', engine, schema='schema_snow', if_exists='append', index=False)
display_results(engine, get_random_rows_sql("schema_snow.gyms"))
display_count("csv_snow", "schema_snow.gyms")

# load checkins table
df = pd.read_sql_query(checkins_load_snow_sql, con=engine_src)
df.to_sql('checkins', engine, schema='schema_snow', if_exists='append', index=False)
display_results(engine, get_random_rows_sql("schema_snow.checkins"))
display_count("csv_snow", "schema_snow.checkins")


[(1, 'Basic', Decimal('19.99'), 'Access to basic gym facilities, Limited class access, 1 guest pass per month'), (2, 'Pro', Decimal('49.99'), 'Access to all facilities, Unlimited class access, 5 guest passes per month, Free personal trainer session'), (3, 'Student', Decimal('9.99'), 'Access to basic facilities, Limited class access, 1 guest pass per month, Discounted for students')]
schema_snow.subscription_plans row count: 3
[(4075, 2, 'John', 'Johnson', 47, 'Female', datetime.date(1997, 12, 27), datetime.date(2021, 11, 6), 'Orlando'), (3425, 1, 'Chris', 'Moore', 37, 'Male', datetime.date(1969, 9, 21), datetime.date(2021, 11, 4), 'San Francisco'), (3141, 3, 'Jessica', 'Wilson', 26, 'Female', datetime.date(1988, 6, 29), datetime.date(2022, 5, 9), 'San Francisco')]
schema_snow.users row count: 5000
[(9, 'Dallas', 'Premium', 'Sauna, CrossFit, Yoga Classes'), (5, 'Phoenix', 'Standard', 'Basketball Court, CrossFit, Swimming Pool'), (6, 'Philadelphia', 'Budget', 'Swimming Pool, Climbing Wal

The databases are now successfully populated with data and are ready for querying to extract insights. Given that the initial datasets used for filling the tables are relatively modest in size, the creation of database indexes is not a necessity at this stage. Nevertheless, it's important to note that as the volume of data increases, the efficiency of queries may diminish. Therefore, it's advisable to establish indexes on foreign key constraints as a standard practice to maintain query performance over time.

```sql
-- OLTP indexes
CREATE INDEX checkins_idx01 ON schema_oltp.checkins(fk_user_id);
CREATE INDEX checkins_idx02 ON schema_oltp.checkins(fk_gym_id);

CREATE INDEX subscription_plans_idx01 ON schema_oltp.users(fk_subscription_plan_id);

-- OLAP snowflake indexes
CREATE INDEX checkins_idx01 ON schema_snow.checkins(fk_user_id);
CREATE INDEX checkins_idx02 ON schema_snow.checkins(fk_gym_id);

CREATE INDEX subscription_plans_idx01 ON schema_snow.users(fk_subscription_plan_id);

```